### imports

In [1]:
import re
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
from datetime import date
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import statsmodels.formula.api as sm

from plotly import tools
from bs4 import BeautifulSoup
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline  

In [2]:
from time import time

import xgboost

from keras.models import Sequential
from keras.layers import Dense, Activation

from sklearn.metrics import r2_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

Using TensorFlow backend.


In [22]:
data = pd.read_csv('projectdata.csv')
data = data.dropna() #For later use in case of bad row
data = data.reset_index(drop=True)
len(data)

59655

### Functions

In [4]:
df_Teams = pd.read_csv('df_Teams.csv')
df_Referee = pd.read_csv('df_Referee.csv')

In [5]:
def addTeam(team):
    global df_Teams
    
    df_Teams = df_Teams.append({"team":team 
                 ,"H_Games":0 ,"H_FTWins":0 ,"H_FTDraws":0 ,"H_FTLost":0 ,"H_HTWins":0 ,"H_HTDraws":0 ,"H_HTLost":0 
                 ,"H_FTGF":0 ,"H_FTGA":0 ,"H_FTGD":0 ,"H_HTGF":0 ,"H_HTGA":0 ,"H_HTGD":0 ,"H_SF":0 ,"H_SA":0 
                 ,"H_SoTF":0 ,"H_SoTA":0 ,"H_CF":0 ,'H_CA':0 ,"H_FF":0 ,"H_FA":0 ,"H_YCF":0 ,"H_YCA":0 ,"H_RCF":0 
                 ,"H_RCA":0 

                 ,"A_Games":0 ,"A_FTWins":0 ,"A_FTDraws":0 ,"A_FTLost":0 ,"A_HTWins":0 ,"A_HTDraws":0 ,"A_HTLost":0 
                 ,"A_FTGF":0 ,"A_FTGA":0 ,"A_FTGD":0 ,"A_HTGF":0 ,"A_HTGA":0 ,"A_HTGD":0 ,"A_SF":0 ,"A_SA":0 
                 ,"A_SoTF":0 ,"A_SoTA":0 ,"A_CF":0 ,'A_CA':0 ,"A_FF":0 ,"A_FA":0 ,"A_YCF":0 ,"A_YCA":0 ,"A_RCF":0 
                 ,"A_RCA":0 
                }, ignore_index=True)

In [6]:
def addRef(Ref):
    global df_Referee

    df_Referee = df_Referee.append({"Name":Ref ,"Games":0 ,"YD":0 ,"RD":0 ,"FKC":0 ,"YPG":0 ,"RPG":0 ,"FKCPG":0
                    }, ignore_index=True)

In [7]:
def updateTeamsTable(Side, OtherSide, row, team):
    global df_Teams
    
    FTHG = row['FT' + Side +'G']
    FTAG = row['FT' + OtherSide +'G']
    FTR = row['FTR']
    HTHG = row['HT' + Side +'G']
    HTAG = row['HT' + OtherSide +'G']
    HTR = row['HTR']
    HS = row[Side +'S']
    AS = row[OtherSide +'S']
    HST = row[Side +'ST']
    AST = row[OtherSide +'ST']
    HF = row[Side +'F']
    AF = row[OtherSide +'F']
    HC = row[Side +'C']
    AC = row[OtherSide +'C']
    HY = row[Side +'Y']
    AY = row[OtherSide +'Y']
    HR = row[Side +'R']
    AR = row[OtherSide +'R']

    df_Teams.loc[df_Teams['team'] == team, [Side + '_Games']] = df_Teams[df_Teams['team'] == team][Side + '_Games'].values[0] + 1
    if FTR == Side:
        df_Teams.loc[df_Teams['team'] == team, [Side + '_FTWins']] = df_Teams[df_Teams['team'] == team][Side + '_FTWins'].values[0] + 1 #Full Time Wins
    elif FTR == 'D':
        df_Teams.loc[df_Teams['team'] == team, [Side + '_FTDraws']] = df_Teams[df_Teams['team'] == team][Side + '_FTDraws'].values[0] + 1 #Full Time Draws
    else:
        df_Teams.loc[df_Teams['team'] == team, [Side + '_FTLost']] = df_Teams[df_Teams['team'] == team][Side + '_FTLost'].values[0] + 1 #Full Time Lost
    if HTR == Side:     
        df_Teams.loc[df_Teams['team'] == team, [Side + '_HTWins']] = df_Teams[df_Teams['team'] == team][Side + '_HTWins'].values[0] + 1 #Half Time Wins
    elif HTR == 'D':   
        df_Teams.loc[df_Teams['team'] == team, [Side + '_HTDraws']] = df_Teams[df_Teams['team'] == team][Side + '_HTDraws'].values[0] + 1 #Half Time Draws
    else:   
        df_Teams.loc[df_Teams['team'] == team, [Side + '_HTLost']] = df_Teams[df_Teams['team'] == team][Side + '_HTLost'].values[0] + 1 #Half Time Lost
    df_Teams.loc[df_Teams['team'] == team, [Side + '_FTGF']] = df_Teams[df_Teams['team'] == team][Side + '_FTGF'].values[0] +  FTHG #Full Time Goals For
    df_Teams.loc[df_Teams['team'] == team, [Side + '_FTGA']] = df_Teams[df_Teams['team'] == team][Side + '_FTGA'].values[0] +  FTAG #Full Time Goals Agienst
    df_Teams.loc[df_Teams['team'] == team, [Side + '_FTGD']] = df_Teams[df_Teams['team'] == team][Side + '_FTGF'].values[0] -  df_Teams[df_Teams['team'] == team][Side + '_FTGA'].values[0] #Full Time Goal Diffence
    df_Teams.loc[df_Teams['team'] == team, [Side + '_HTGF']] = df_Teams[df_Teams['team'] == team][Side + '_HTGF'].values[0] + HTHG #Half Time Goals For
    df_Teams.loc[df_Teams['team'] == team, [Side + '_HTGA']] = df_Teams[df_Teams['team'] == team][Side + '_HTGA'].values[0] + HTAG #Half Time Goals Agienst
    df_Teams.loc[df_Teams['team'] == team, [Side + '_HTGD']] = df_Teams[df_Teams['team'] == team][Side + '_HTGF'].values[0] -  df_Teams[df_Teams['team'] == team][Side + '_HTGA'].values[0]#Half Time Goal Diffence
    df_Teams.loc[df_Teams['team'] == team, [Side + '_SF']] = df_Teams[df_Teams['team'] == team][Side + '_SF'].values[0] + HS #Shots For
    df_Teams.loc[df_Teams['team'] == team, [Side + '_SA']] = df_Teams[df_Teams['team'] == team][Side + '_SA'].values[0] + AS #Shots Agienst
    df_Teams.loc[df_Teams['team'] == team, [Side + '_SoTF']] = df_Teams[df_Teams['team'] == team][Side + '_SoTF'].values[0] + HST #Shots on Target For
    df_Teams.loc[df_Teams['team'] == team, [Side + '_SoTA']] = df_Teams[df_Teams['team'] == team][Side + '_SoTA'].values[0] + AST #Shots on Target Agienst
    df_Teams.loc[df_Teams['team'] == team, [Side + '_CF']] = df_Teams[df_Teams['team'] == team][Side + '_CF'].values[0] + HC #Corners For
    df_Teams.loc[df_Teams['team'] == team, [Side + '_CA']] = df_Teams[df_Teams['team'] == team][Side + '_CA'].values[0] + AC #Corners Agienst
    df_Teams.loc[df_Teams['team'] == team, [Side + '_FF']] = df_Teams[df_Teams['team'] == team][Side + '_FF'].values[0] + HF #Fouls Committed by rival
    df_Teams.loc[df_Teams['team'] == team, [Side + '_FA']] = df_Teams[df_Teams['team'] == team][Side + '_FA'].values[0] + AF #Fouls Committed by the team
    df_Teams.loc[df_Teams['team'] == team, [Side + '_YCF']] = df_Teams[df_Teams['team'] == team][Side + '_YCF'].values[0] + HY #Yellow Cards For
    df_Teams.loc[df_Teams['team'] == team, [Side + '_YCA']] = df_Teams[df_Teams['team'] == team][Side + '_YCA'].values[0] + AY #Yellow Cards Agienst
    df_Teams.loc[df_Teams['team'] == team, [Side + '_RCF']] = df_Teams[df_Teams['team'] == team][Side + '_RCF'].values[0] + HR #Red Cards For
    df_Teams.loc[df_Teams['team'] == team, [Side + '_RCA']] = df_Teams[df_Teams['team'] == team][Side + '_RCA'].values[0] + AR #Red Cards Agienst 

In [8]:
def updateRefTable(Ref, HY, AY, HR, AR, HF, AF):
    global df_Referee
    
    RG, YP, RP, FKC = getRefStats(Ref)
    
    df_Referee.loc[df_Referee['Name'] == Ref,['Games']] = RG + 1
    df_Referee.loc[df_Referee['Name'] == Ref,['YD']] = YP + HY + AY #Yellow Drawn
    df_Referee.loc[df_Referee['Name'] == Ref,['RD']] = RP + HR + AR #Red Drawn
    df_Referee.loc[df_Referee['Name'] == Ref,['FKC']] = FKC + HF + AF #Free kicks Calls
    df_Referee.loc[df_Referee['Name'] == Ref,['YPG']] = df_Referee[df_Referee['Name'] == Ref]['YD'].values[0] / (RG + 1)#Yellow Per Game
    df_Referee.loc[df_Referee['Name'] == Ref,['RPG']] = df_Referee[df_Referee['Name'] == Ref]['RD'].values[0] / (RG + 1) #Red Per Game
    df_Referee.loc[df_Referee['Name'] == Ref,['FKCPG']] = df_Referee[df_Referee['Name'] == Ref]['FKC'].values[0] / (RG + 1) #Free kicks Calls Per Game
    

In [9]:
def updateDataTable(HT, AT, Ref, row):
    global data
    
    #Each side data
    HGames, H_FTWPG, H_FTDPG, H_FTLPG, H_HTWPG, H_HTDPG, H_HTLPG, H_FTGFPG, H_FTGAPG, H_HTGFPG, H_HTGAPG, H_SFPG, H_SAPG, H_SoTFPG, H_SoTAPG, H_CFPG, H_CAPG, H_FFPG, H_FAPG, H_YCFPG, H_YCAPG, H_RCFPG, H_RCAPG, AGames, A_FTWPG, A_FTDPG, A_FTLPG, A_HTWPG, A_HTDPG, A_HTLPG, A_FTGFPG, A_FTGAPG, A_HTGFPG, A_HTGAPG, A_SFPG, A_SAPG, A_SoTFPG, A_SoTAPG, A_CFPG, A_CAPG, A_FFPG, A_FAPG, A_YCFPG, A_YCAPG, A_RCFPG, A_RCAPG = getTeamsStats(HT, AT)
    
    #For total data
    HGames2, H_FTWPG2, H_FTDPG2, H_FTLPG2, H_HTWPG2, H_HTDPG2, H_HTLPG2, H_FTGFPG2, H_FTGAPG2, H_HTGFPG2, H_HTGAPG2, H_SFPG2, H_SAPG2, H_SoTFPG2, H_SoTAPG2, H_CFPG2, H_CAPG2, H_FFPG2, H_FAPG2, H_YCFPG2, H_YCAPG2, H_RCFPG2, H_RCAPG2, AGames2,  A_FTWPG2, A_FTDPG2, A_FTLPG2, A_HTWPG2, A_HTDPG2, A_HTLPG2, A_FTGFPG2, A_FTGAPG2, A_HTGFPG2, A_HTGAPG2, A_SFPG2, A_SAPG2, A_SoTFPG2, A_SoTAPG2, A_CFPG2, A_CAPG2, A_FFPG2, A_FAPG2, A_YCFPG2, A_YCAPG2, A_RCFPG2, A_RCAPG2 = getTeamsStats(AT, HT)
    
    #Previues Result for both teams and Previues Encounter
    HPR1, HPR2, HPR3, HPR4, HPR5 = getLastFiveGames(HT, 'Home')
    APR1, APR2, APR3, APR4, APR5 = getLastFiveGames(AT, 'Away')
    PE1, PE2, PE3, PE4, PE5 = getLastFiveEncounter(HT,AT)
    
    #Ref stats
    #RG, YP, RP, FKC = getRefStats(Ref)
    
    #In case games is zero and we cant divide
    if HGames == 0:
        HomeCorrection = 0
        HGames = 1
    else:
        HomeCorrection = HGames
    if AGames == 0:
        AwayCorrection = 0
        AGames = 1
    else:
        AwayCorrection = AGames
        
    if HGames2 == 0:
        HomeCorrection2 = 0
        HGames2 = 1
    else:
        HomeCorrection2 = HGames2
    if AGames2 == 0:
        AwayCorrection2 = 0
        AGames2 = 1
    else:
        AwayCorrection2 = AGames2
        
    RGC = 1
    #if RG > 0:      
        #RGC = RG
    
    data = data.append({'Date':row['Date'],'HomeTeam':HT,'AwayTeam':AT,'FTGD':(row['FTHG'] - row['FTAG']) 
             ,'FTR':row['FTR'] ,'HTGD':row['HTHG'] - row['HTAG'] ,'HTR':row['HTR'] #,'Referee':Ref
 
                #Total stats - Home Team Total(Both home and away history stats)
             ,'TH_Games':HomeCorrection + AwayCorrection2        
             ,'TH_FTW':(H_FTWPG + A_FTWPG2) #Total FT Win
             ,'TH_FTD':(H_FTDPG + A_FTDPG2) #Total FT Draw
             ,'TH_FTL':(H_FTLPG + A_FTLPG2) #Total FT Loss
             ,'TH_HTW':(H_HTWPG + A_HTWPG2) #Total HT Win
             ,'TH_HTD':(H_HTDPG + A_HTDPG2) #Total HT Draw
             ,'TH_HTL':(H_HTLPG + A_HTLPG2) #Total HT Loss
             ,'TH_FTGF':(H_FTGFPG + A_FTGFPG2) #Total FT Goal For
             ,'TH_FTGA':(H_FTGAPG + A_FTGAPG2) #Total FT Goal Agienst
             ,'TH_FTGD':((H_FTGFPG - H_FTGAPG) + (A_FTGFPG2 - A_FTGAPG2)) #Total FT Goal Diffrence
             ,'TH_HTGF':(H_HTGFPG + A_HTGFPG2) #Total HT Goal For
             ,'TH_HTGA':(H_HTGAPG + A_HTGAPG2) #Total HT Goal Agienst
             ,'TH_HTGD':((H_HTGFPG - H_HTGAPG) + (A_HTGFPG2 - A_HTGAPG2)) #Total HT Goal Diffrence
             ,'TH_SF':(H_SFPG + A_SFPG2) #Total Shot For
             ,'TH_SA':(H_SAPG + A_SAPG2) #Total Shot Agienst
             ,'TH_SoTF':(H_SoTFPG + A_SoTFPG2) #Total Shot on Target For
             ,'TH_SoTA':(H_SoTAPG + A_SoTAPG2) #Total Shot on Target Agienst
             ,'TH_CF':(H_CFPG + A_CFPG2) #Total Corners For
             ,'TH_CA':(H_CAPG + A_CAPG2) #Total Corners Agienst
             ,'TH_FF':(H_FFPG + A_FFPG2) #Total Fouls For 
             ,'TH_FA':(H_FAPG + A_FAPG2) #Total Fouls Agienst
             ,'TH_YCF':(H_YCFPG + A_YCFPG2) #Total Yellow Card For
             ,'TH_YCA':(H_YCAPG + A_YCAPG2) #Total Yellow Card Agienst
             ,'TH_RCF':(H_RCFPG + A_RCFPG2) #Total Red Card For
             ,'TH_RCA':(H_RCAPG + A_RCAPG2) #Total Red Card Agienst     
                        
                #Per Game stats - Home Team Total(Both home and away history stats)      
             ,'TH_FTWPG':(H_FTWPG + A_FTWPG2) / (HGames + AGames2) #Total FT Win Per Game
             ,'TH_FTDPG':(H_FTDPG + A_FTDPG2) / (HGames + AGames2) #Total FT Draw Per Game 
             ,'TH_FTLPG':(H_FTLPG + A_FTLPG2) / (HGames + AGames2) #Total FT Loss Per Game
             ,'TH_HTWPG':(H_HTWPG + A_HTWPG2) / (HGames + AGames2) #Total HT Win Per Game
             ,'TH_HTDPG':(H_HTDPG + A_HTDPG2) / (HGames + AGames2) #Total HT Draw Per Game
             ,'TH_HTLPG':(H_HTLPG + A_HTLPG2) / (HGames + AGames2) #Total HT Loss Per Game
             ,'TH_FTGFPG':(H_FTGFPG + A_FTGFPG2) / (HGames + AGames2) #Total FT Goal For
             ,'TH_FTGAPG':(H_FTGAPG + A_FTGAPG2) / (HGames + AGames2) #Total FT Goal Agienst
             ,'TH_FTGDPG':((H_FTGFPG - H_FTGAPG) + (A_FTGFPG2 - A_FTGAPG2)) / (HGames + AGames2) #Total FT Goal Diffrence
             ,'TH_HTGFPG':(H_HTGFPG + A_HTGFPG2) / (HGames + AGames2) #Total HT Goal For
             ,'TH_HTGAPG':(H_HTGAPG + A_HTGAPG2) / (HGames + AGames2) #Total HT Goal Agienst
             ,'TH_HTGDPG':((H_HTGFPG - H_HTGAPG) + (A_HTGFPG2 - A_HTGAPG2)) / (HGames + AGames2) #Total HT Goal Diffrence
             ,'TH_SFPG':(H_SFPG + A_SFPG2) / (HGames + AGames2) #Total Shot For Per Game
             ,'TH_SAPG':(H_SAPG + A_SAPG2) / (HGames + AGames2) #Total Shot Agienst Per Game
             ,'TH_SoTFPG':(H_SoTFPG + A_SoTFPG2) / (HGames + AGames2) #Total Shot on Target For Per Game
             ,'TH_SoTAPG':(H_SoTAPG + A_SoTAPG2) / (HGames + AGames2) #Total Shot on Target Agienst Per Game
             ,'TH_CFPG':(H_CFPG + A_CFPG2) / (HGames + AGames2) #Total Corners For Per Game 
             ,'TH_CAPG':(H_CAPG + A_CAPG2) / (HGames + AGames2) #Total Corners Agienst Per Game 
             ,'TH_FFPG':(H_FFPG + A_FFPG2) / (HGames + AGames2) #Total Fouls For Per Game 
             ,'TH_FAPG':(H_FAPG + A_FAPG2) / (HGames + AGames2) #Total Fouls Agienst Per Game 
             ,'TH_YCFPG':(H_YCFPG + A_YCFPG2) / (HGames + AGames2) #Total Yellow Card For Per Game
             ,'TH_YCAPG':(H_YCAPG + A_YCAPG2) / (HGames + AGames2) #Total Yellow Card Agienst Per Game
             ,'TH_RCFPG':(H_RCFPG + A_RCFPG2) / (HGames + AGames2) #Total Red Card For Per Game
             ,'TH_RCAPG':(H_RCAPG + A_RCAPG2) / (HGames + AGames2) #Total Red Card Agienst Per Game           
               
             #Side stats - Home Team
             ,'H_Games':HomeCorrection               
             ,'H_FTW':H_FTWPG #FT Win 
             ,'H_FTD':H_FTDPG #FT Draw  
             ,'H_FTL':H_FTLPG #FT Loss 
             ,'H_HTW':H_HTWPG #HT Win 
             ,'H_HTD':H_HTDPG #HT Draw 
             ,'H_HTL':H_HTLPG #HT Loss 
             ,'H_FTGF':H_FTGFPG #FT Goal For
             ,'H_FTGA':H_FTGAPG #FT Goal Agienst
             ,'H_FTGD':(H_FTGFPG - H_FTGAPG) #FT Goal Diffrence
             ,'H_HTGF':H_HTGFPG #HT Goal For
             ,'H_HTGA':H_HTGAPG #HT Goal Agienst
             ,'H_HTGD':(H_HTGFPG - H_HTGAPG) #HT Goal Diffrence
             ,'H_SF':H_SFPG #Shot For 
             ,'H_SA':H_SAPG #Shot Agienst 
             ,'H_SoTF':H_SoTFPG #Shot on Target For 
             ,'H_SoTA':H_SoTAPG #Shot on Target Agienst
             ,'H_CF':H_CFPG #Corners For 
             ,'H_CA':H_CAPG #Corners Agienst 
             ,'H_FF':H_FFPG #Fouls For 
             ,'H_FA':H_FAPG #Fouls Agienst 
             ,'H_YCF':H_YCFPG #Yellow Card For 
             ,'H_YCA':H_YCAPG #Yellow Card Agienst 
             ,'H_RCF':H_RCFPG #Red Card For 
             ,'H_RCA':H_RCAPG #Red Card Agienst 
                        
             #Per Game stats - Home Team              
             ,'H_FTWPG':H_FTWPG / HGames #FT Win Per Game
             ,'H_FTDPG':H_FTDPG / HGames #FT Draw Per Game 
             ,'H_FTLPG':H_FTLPG / HGames #FT Loss Per Game
             ,'H_HTWPG':H_HTWPG / HGames #HT Win Per Game
             ,'H_HTDPG':H_HTDPG / HGames #HT Draw Per Game
             ,'H_HTLPG':H_HTLPG / HGames #HT Loss Per Game
             ,'H_FTGFPG':H_FTGFPG / HGames #FT Goal For
             ,'H_FTGAPG':H_FTGAPG / HGames #FT Goal Agienst
             ,'H_FTGDPG':(H_FTGFPG - H_FTGAPG) / HGames #FT Goal Diffrence
             ,'H_HTGFPG':H_HTGFPG / HGames #HT Goal For
             ,'H_HTGAPG':H_HTGAPG / HGames #HT Goal Agienst
             ,'H_HTGDPG':(H_HTGFPG - H_HTGAPG) / HGames #HT Goal Diffrence
             ,'H_SFPG':H_SFPG / HGames #Shot For Per Game
             ,'H_SAPG':H_SAPG / HGames #Shot Agienst Per Game
             ,'H_SoTFPG':H_SoTFPG / HGames #Shot on Target For Per Game
             ,'H_SoTAPG':H_SoTAPG / HGames #Shot on Target Agienst Per Game
             ,'H_CFPG':H_CFPG / HGames #Corners For Per Game 
             ,'H_CAPG':H_CAPG / HGames #Corners Agienst Per Game 
             ,'H_FFPG':H_FFPG / HGames #Fouls For Per Game 
             ,'H_FAPG':H_FAPG / HGames #Fouls Agienst Per Game 
             ,'H_YCFPG':H_YCFPG / HGames #Yellow Card For Per Game
             ,'H_YCAPG':H_YCAPG / HGames #Yellow Card Agienst Per Game
             ,'H_RCFPG':H_RCFPG / HGames #Red Card For Per Game
             ,'H_RCAPG':H_RCAPG / HGames #Red Card Agienst Per Game

                #Total stats - Away Team Total(Both home and away history stats) 
             ,'TA_Games':HomeCorrection2 + AwayCorrection        
             ,'TA_FTW':(H_FTWPG2 + A_FTWPG) #Total FT Win
             ,'TA_FTD':(H_FTDPG2 + A_FTDPG) #Total FT Draw
             ,'TA_FTL':(H_FTLPG2 + A_FTLPG) #Total FT Loss
             ,'TA_HTW':(H_HTWPG2 + A_HTWPG) #Total HT Win
             ,'TA_HTD':(H_HTDPG2 + A_HTDPG) #Total HT Draw
             ,'TA_HTL':(H_HTLPG2 + A_HTLPG) #Total HT Loss
             ,'TA_FTGF':(H_FTGFPG2 + A_FTGFPG) #Total FT Goal For
             ,'TA_FTGA':(H_FTGAPG2 + A_FTGAPG) #Total FT Goal Agienst
             ,'TA_FTGD':((H_FTGFPG2 - H_FTGAPG2) + (A_FTGFPG - A_FTGAPG)) #Total FT Goal Diffrence
             ,'TA_HTGF':(H_HTGFPG2 + A_HTGFPG) #Total HT Goal For
             ,'TA_HTGA':(H_HTGAPG2 + A_HTGAPG) #Total HT Goal Agienst
             ,'TA_HTGD':((H_HTGFPG2 - H_HTGAPG2) + (A_HTGFPG - A_HTGAPG)) #Total HT Goal Diffrence
             ,'TA_SF':(H_SFPG2 + A_SFPG) #Total Shot For
             ,'TA_SA':(H_SAPG2 + A_SAPG) #Total Shot Agienst
             ,'TA_SoTF':(H_SoTFPG2 + A_SoTFPG) #Total Shot on Target For
             ,'TA_SoTA':(H_SoTAPG2 + A_SoTAPG) #Total Shot on Target Agienst
             ,'TA_CF':(H_CFPG2 + A_CFPG) #Total Corners For
             ,'TA_CA':(H_CAPG2 + A_CAPG) #Total Corners Agienst
             ,'TA_FF':(H_FFPG2 + A_FFPG) #Total Fouls For 
             ,'TA_FA':(H_FAPG2 + A_FAPG) #Total Fouls Agienst
             ,'TA_YCF':(H_YCFPG2 + A_YCFPG) #Total Yellow Card For
             ,'TA_YCA':(H_YCAPG2 + A_YCAPG) #Total Yellow Card Agienst
             ,'TA_RCF':(H_RCFPG2 + A_RCFPG) #Total Red Card For
             ,'TA_RCA':(H_RCAPG2 + A_RCAPG) #Total Red Card Agienst     
                        
                #Per Game stats - Away Team Total(Both home and away history stats)   
             ,'TA_FTWPG':(H_FTWPG2 + A_FTWPG) / (HGames2 + AGames) #Total FT Win Per Game
             ,'TA_FTDPG':(H_FTDPG2 + A_FTDPG) / (HGames2 + AGames) #Total FT Draw Per Game 
             ,'TA_FTLPG':(H_FTLPG2 + A_FTLPG) / (HGames2 + AGames) #Total FT Loss Per Game
             ,'TA_HTWPG':(H_HTWPG2 + A_HTWPG) / (HGames2 + AGames) #Total HT Win Per Game
             ,'TA_HTDPG':(H_HTDPG2 + A_HTDPG) / (HGames2 + AGames) #Total HT Draw Per Game
             ,'TA_HTLPG':(H_HTLPG2 + A_HTLPG) / (HGames2 + AGames) #Total HT Loss Per Game
             ,'TA_FTGFPG':(H_FTGFPG2 + A_FTGFPG) / (HGames2 + AGames) #Total FT Goal For
             ,'TA_FTGAPG':(H_FTGAPG2 + A_FTGAPG) / (HGames2 + AGames) #Total FT Goal Agienst
             ,'TA_FTGDPG':((H_FTGFPG2 - H_FTGAPG2) + (A_FTGFPG - A_FTGAPG)) / (HGames2 + AGames) #Total FT Goal Diffrence
             ,'TA_HTGFPG':(H_HTGFPG2 + A_HTGFPG) / (HGames2 + AGames) #Total HT Goal For
             ,'TA_HTGAPG':(H_HTGAPG2 + A_HTGAPG) / (HGames2 + AGames) #Total HT Goal Agienst
             ,'TA_HTGDPG':((H_HTGFPG2 - H_HTGAPG2) + (A_HTGFPG - A_HTGAPG)) / (HGames2 + AGames) #Total HT Goal Diffrence
             ,'TA_SFPG':(H_SFPG2 + A_SFPG) / (HGames2 + AGames) #Total Shot For Per Game
             ,'TA_SAPG':(H_SAPG2 + A_SAPG) / (HGames2 + AGames) #Total Shot Agienst Per Game
             ,'TA_SoTFPG':(H_SoTFPG2 + A_SoTFPG) / (HGames2 + AGames) #Total Shot on Target For Per Game
             ,'TA_SoTAPG':(H_SoTAPG2 + A_SoTAPG) / (HGames2 + AGames) #Total Shot on Target Agienst Per Game
             ,'TA_CFPG':(H_CFPG2 + A_CFPG) / (HGames2 + AGames) #Total Corners For Per Game 
             ,'TA_CAPG':(H_CAPG2 + A_CAPG) / (HGames2 + AGames) #Total Corners Agienst Per Game 
             ,'TA_FFPG':(H_FFPG2 + A_FFPG) / (HGames2 + AGames) #Total Fouls For Per Game 
             ,'TA_FAPG':(H_FAPG2 + A_FAPG) / (HGames2 + AGames) #Total Fouls Agienst Per Game 
             ,'TA_YCFPG':(H_YCFPG2 + A_YCFPG) / (HGames2 + AGames) #Total Yellow Card For Per Game
             ,'TA_YCAPG':(H_YCAPG2 + A_YCAPG) / (HGames2 + AGames) #Total Yellow Card Agienst Per Game
             ,'TA_RCFPG':(H_RCFPG2 + A_RCFPG) / (HGames2 + AGames) #Total Red Card For Per Game
             ,'TA_RCAPG':(H_RCAPG2 + A_RCAPG) / (HGames2 + AGames) #Total Red Card Agienst Per Game   
                
             #side stats - Away Team
             ,'A_Games':AwayCorrection
             ,'A_FTW':A_FTWPG #FT Win Per Game
             ,'A_FTD':A_FTWPG #FT Draw Per Game 
             ,'A_FTL':A_FTLPG #FT Loss Per Game
             ,'A_HTW':A_HTWPG #HT Win Per Game
             ,'A_HTD':A_HTDPG #HT Draw Per Game
             ,'A_HTL':A_HTLPG #HT Loss Per Game
             ,'A_FTGF':A_FTGFPG #FT Goal For
             ,'A_FTGA':A_FTGAPG #FT Goal Agienst
             ,'A_FTGD':(A_FTGFPG - A_FTGAPG) #FT Goal Diffrence
             ,'A_HTGF':A_HTGFPG #HT Goal For
             ,'A_HTGA':A_HTGAPG #HT Goal Agienst
             ,'A_HTGD':(A_HTGFPG - A_HTGAPG) #HT Goal Diffrence
             ,'A_SF':A_SFPG #Shot For Per Game
             ,'A_SA':A_SAPG #Shot Agienst Per Game
             ,'A_SoTF':A_SoTFPG #Shot on Target For Per Game
             ,'A_SoTA':A_SoTAPG #Shot on Target Agienst Per Game
             ,'A_CF':A_CFPG #Corners For Per Game 
             ,'A_CA':A_CAPG #Corners Agienst Per Game 
             ,'A_FF':A_FFPG #Fouls For Per Game 
             ,'A_FA':A_FAPG #Fouls Agienst Per Game 
             ,'A_YCF':A_YCFPG #Yellow Card For Per Game
             ,'A_YCA':A_YCAPG #Yellow Card Agienst Per Game
             ,'A_RCF':A_RCFPG #Red Card For Per Game
             ,'A_RCA':A_RCAPG #Red Card Agienst Per Game
                        
             #Per Game stats - Away Team
             ,'A_FTWPG':A_FTWPG / AGames #FT Win Per Game
             ,'A_FTDPG':A_FTWPG / AGames #FT Draw Per Game 
             ,'A_FTLPG':A_FTLPG / AGames #FT Loss Per Game
             ,'A_HTWPG':A_HTWPG / AGames #HT Win Per Game
             ,'A_HTDPG':A_HTDPG / AGames #HT Draw Per Game
             ,'A_HTLPG':A_HTLPG / AGames #HT Loss Per Game
             ,'A_FTGFPG':A_FTGFPG / AGames #FT Goal For
             ,'A_FTGAPG':A_FTGAPG / AGames #FT Goal Agienst
             ,'A_FTGDPG':(A_FTGFPG - A_FTGAPG) / AGames #FT Goal Diffrence
             ,'A_HTGFPG':A_HTGFPG / AGames #HT Goal For
             ,'A_HTGAPG':A_HTGAPG / AGames #HT Goal Agienst
             ,'A_HTGDPG':(A_HTGFPG - A_HTGAPG) / AGames #HT Goal Diffrence
             ,'A_SFPG':A_SFPG / AGames #Shot For Per Game
             ,'A_SAPG':A_SAPG / AGames #Shot Agienst Per Game
             ,'A_SoTFPG':A_SoTFPG / AGames #Shot on Target For Per Game
             ,'A_SoTAPG':A_SoTAPG / AGames #Shot on Target Agienst Per Game
             ,'A_CFPG':A_CFPG / AGames #Corners For Per Game 
             ,'A_CAPG':A_CAPG / AGames #Corners Agienst Per Game 
             ,'A_FFPG':A_FFPG / AGames #Fouls For Per Game 
             ,'A_FAPG':A_FAPG / AGames #Fouls Agienst Per Game 
             ,'A_YCFPG':A_YCFPG / AGames #Yellow Card For Per Game
             ,'A_YCAPG':A_YCAPG / AGames #Yellow Card Agienst Per Game
             ,'A_RCFPG':A_RCFPG / AGames #Red Card For Per Game
             ,'A_RCAPG':A_RCAPG / AGames #Red Card Agienst Per Game

             #HPR = Home Previues Result, APR = Away Previues Result, PE = Previues Encounter
             ,'HPR1':HPR1,'HPR2':HPR2,'HPR3':HPR3,'HPR4':HPR4,'HPR5':HPR5
             ,'APR1':APR1,'APR2':APR2,'APR3':APR3,'APR4':APR4,'APR5':APR5
             ,'PE1':PE1,'PE2':PE2,'PE3':PE3,'PE4':PE4,'PE5':PE5
              
             #Referee Per Game stats
             #,'RefGames':RG, 'RefYD':YP, 'RefRD':RP, 'RefFKC':FKC, 'RefYDPG':YP/RGC, 'RefRDPG':RP/RGC, 'RefFKCPG':FKC/RGC
    
             #Bet stats
             ,'B365H':row['B365H'], 'B365D':row['B365D'], 'B365A':row['B365A'],'BWH':row['BWH'], 'BWD':row['BWD']
             ,'BWA':row['BWA'], 'IWH':row['IWH'], 'IWD':row['IWD'],'IWA':row['IWA'],'PSH':row['PSH'], 'PSD':row['PSD']
             ,'PSA':row['PSA'],'PSCH':row['PSCH'],'PSCD':row['PSCD'], 'PSCA':row['PSCA']
             ,'WHH':row['WHH'], 'WHD':row['WHD'], 'WHA':row['WHA'],  'VCH':row['VCH'], 'VCD':row['VCD'], 'VCA':row['VCA']


            #Other Stats - TODO ??? 
            #df_Teams.team['T'] #Titels
            #df_Teams.team['RU'] #Runner Ups
            #df_Teams.team['CC'] #Current Champions
            #df_Teams.team['CRU'] #Current Runner Up


             }, ignore_index=True) 

In [10]:
def getLastFiveGames(team, side):
    global data
    
    N = 5
    LastFiveHomeGames = data[data['HomeTeam'] == team][-N:]
    LastFiveAwayGames = data[data['AwayTeam'] == team][-N:]
    
    LastFiveHomeGames = LastFiveHomeGames.append(LastFiveAwayGames)
    LastFiveHomeGames = LastFiveHomeGames.sort_index()
    
    LastFiveHomeGames = LastFiveHomeGames[-N:]
    if len(LastFiveHomeGames) > 4:
        PR1 = LastFiveHomeGames["FTGD"].iloc[4]
    else:
        PR1 = 0
    if len(LastFiveHomeGames) > 3:
        PR2 = LastFiveHomeGames["FTGD"].iloc[3]
    else:
        PR2 = 0
    if len(LastFiveHomeGames) > 2:
        PR3 = LastFiveHomeGames["FTGD"].iloc[2]
    else:
        PR3 = 0
    if len(LastFiveHomeGames) > 1:
        PR4 = LastFiveHomeGames["FTGD"].iloc[1]
    else:
        PR4 = 0
    if len(LastFiveHomeGames) > 0:
        PR5 = LastFiveHomeGames["FTGD"].iloc[0]
    else:
        PR5 = 0

    listofPR = [PR1, PR2, PR3, PR4, PR5]
    for index in range(len(listofPR)):
        if len(LastFiveHomeGames) > (4 - index):
            if ((side == 'Home' and LastFiveHomeGames["HomeTeam"].iloc[4 - index] != team) or
                (side == 'Away' and LastFiveHomeGames["AwayTeam"].iloc[4 - index] != team)):
                listofPR[index] = listofPR[index] * (-1)
    
    return listofPR[0], listofPR[1], listofPR[2], listofPR[3], listofPR[4]

In [11]:
def getLastFiveEncounter(HT,AT):
    global data
    
    N = 5
    LastFiveHomeGames = data.loc[data['HomeTeam'] == HT]
    LastFiveHomeGames = LastFiveHomeGames.loc[LastFiveHomeGames['AwayTeam'] == AT][-N:]
    
    LastFiveAwayGames = data.loc[data['HomeTeam'] == AT]
    LastFiveAwayGames = LastFiveAwayGames.loc[LastFiveAwayGames['AwayTeam'] == HT][-N:]
    
    LastFiveHomeGames = LastFiveHomeGames.append(LastFiveAwayGames)
    LastFiveHomeGames = LastFiveHomeGames.sort_index()
    
    LastFiveHomeGames = LastFiveHomeGames[-N:]
    if len(LastFiveHomeGames) > 4:
        PE1 = LastFiveHomeGames["FTGD"].iloc[4]
    else:
        PE1 = 0
    if len(LastFiveHomeGames) > 3:
        PE2 = LastFiveHomeGames["FTGD"].iloc[3]
    else:
        PE2 = 0
    if len(LastFiveHomeGames) > 2:
        PE3 = LastFiveHomeGames["FTGD"].iloc[2]
    else:
        PE3 = 0
    if len(LastFiveHomeGames) > 1:
        PE4 = LastFiveHomeGames["FTGD"].iloc[1]
    else:
        PE4 = 0
    if len(LastFiveHomeGames) > 0:
        PE5 = LastFiveHomeGames["FTGD"].iloc[0]
    else:
        PE5 = 0
    
    listofPE = [PE1, PE2, PE3, PE4, PE5]
    for index in range(len(listofPE)):
        if len(LastFiveHomeGames) > (4 - index):
            if LastFiveHomeGames['HomeTeam'].iloc[4 - index] == AT:
                listofPE[index] = listofPE[index] * (-1)
    
    return listofPE[0], listofPE[1], listofPE[2], listofPE[3], listofPE[4]

In [12]:
def getTeamsStats(HT, AT):
    global df_Teams

    return (df_Teams[df_Teams['team'] == HT]['H_Games'].values[0],
            
            df_Teams[df_Teams['team'] == HT]['H_FTWins'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_FTDraws'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_FTLost'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_HTWins'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_HTDraws'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_HTLost'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_FTGF'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_FTGA'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_HTGF'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_HTGA'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_SF'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_SA'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_SoTF'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_SoTA'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_CF'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_CA'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_FF'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_FA'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_YCF'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_YCA'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_RCF'].values[0],
            df_Teams[df_Teams['team'] == HT]['H_RCA'].values[0],
            df_Teams[df_Teams['team'] == HT]['A_Games'].values[0],
            
            df_Teams[df_Teams['team'] == AT]['A_FTWins'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_FTDraws'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_FTLost'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_HTWins'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_HTDraws'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_HTLost'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_FTGF'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_FTGA'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_HTGF'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_HTGA'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_SF'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_SA'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_SoTF'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_SoTA'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_CF'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_CA'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_FF'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_FA'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_YCF'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_YCA'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_RCF'].values[0],
            df_Teams[df_Teams['team'] == AT]['A_RCA'].values[0])

In [13]:
def getRefStats(name):
    global df_Referee
    
    Games = df_Referee[df_Referee['Name'] == name]['Games'].values[0]
    YD = df_Referee[df_Referee['Name'] == name]['YD'].values[0]
    RD = df_Referee[df_Referee['Name'] == name]['RD'].values[0]
    FKC = df_Referee[df_Referee['Name'] == name]['FKC'].values[0]
    
    return Games, YD, RD, FKC

In [14]:
def games(df):
    
    t1=time()
    for index, row in df.iterrows():
        #print('Game ',index)
        t2=time()
        HT = row['HomeTeam']
        AT = row['AwayTeam']
        Ref = 'NoRef' # row["Referee"]
        
        #Add new team - take 0.01 sec
        for team in HT, AT:
            if df_Teams[df_Teams['team'] == team].empty: #Add new team to table
                addTeam (team)
                
        #if df_Referee[df_Referee['Name'] == Ref].empty: #Add new Referee to table
            #addRef(Ref)

        #Update Data Table - take 0.1 sec
        updateDataTable(HT, AT, Ref, row)

        #Update Referee Table
        #updateRefTable(Ref, row['HY'], row['AY'], row['HR'], row['AR'], row['HF'], row['AF'])

        #Update Teams Table - takes 0.1 sec
        for team in HT, AT:
            if team == HT: #Home team
                Side = 'H'
                OtherSide = 'A'
            else: #Away team
                Side = 'A'
                OtherSide = 'H'

            updateTeamsTable(Side, OtherSide, row, team)
            
    #    print("Finished in ",time()-t2," seconds")
    #print((time()-t1)/360,' Hours')

In [15]:
def getNewData(newGames):
    global data
    
    K = len(newGames)
    N = len(data) - K
    newData = data.iloc[N:]
    newData = newData.reset_index(drop=True)
    data = data[:N]
    return newData

### Add new Games

In [16]:
addNewGame = False
toSave = False
if addNewGame:
    LastWeekGames = 'LastWeekGames.csv'
    newFixtures = pd.read_csv(LastWeekGames)
    DivList = ['E0', 'E1', 'F1', 'F2', 'D1', 'D2', 'I1', 'I2', 'SP1', 'SP2']
    for div in DivList:
        newGames = newFixtures[newFixtures['Div'] == div]
        newGames = newGames.fillna(0)
        games(newGames)
    if toSave:
        data.to_csv("projectdata.csv", index=False, sep=',')

In [49]:
time_format = '%d/%m/%Y'
sortData = data.iloc[pd.to_datetime(data.Date, format=time_format).values.argsort()]
sortData = sortData.reset_index(drop=True)
sortData = sortData[10000:].reset_index(drop=True)

### Training

In [50]:
dropBeforTrain = ['Date','HomeTeam','AwayTeam','FTR','HTGD','HTR'
                 ]
train = sortData.drop(dropBeforTrain, axis=1)
train.apply(pd.to_numeric)
train= train.fillna(0)
train.head()

,FTGD,TH_Games,TH_FTW,TH_FTD,TH_FTL,TH_HTW,TH_HTD,TH_HTL,TH_FTGF,TH_FTGA,...,WHA,VCH,VCD,VCA,PSH,PSD,PSA,PSCH,PSCD,PSCA
0,-1,210,53,65,122,46,105,89,233,385,...,2.7,2.30,3.25,2.7,0.0,0.0,0.0,0.0,0.0,0.0
1,0,389,142,97,172,103,182,126,512,554,...,4.5,1.60,3.40,5.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,122,54,28,40,40,45,37,221,169,...,6.0,1.45,3.50,6.5,0.0,0.0,0.0,0.0,0.0,0.0
3,0,319,176,150,225,137,242,172,582,731,...,3.1,1.90,3.25,3.5,0.0,0.0,0.0,0.0,0.0,0.0
4,0,99,36,33,28,27,41,29,131,114,...,4.4,1.75,2.90,5.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
betColumns = train.columns[-21:]
train = train.drop(betColumns, axis=1)

In [53]:
target = 'FTGD'
X = train.drop(target, axis=1).values
y = train[target].values
testSize = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize, random_state=None)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=testSize, random_state=None)

In [54]:
t1 = time()
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

datetime_object = datetime.datetime.now()
print(datetime_object)

XGBC = XGBClassifier()
XGBC.fit(X_train, y_train)

# make predictions for test data
y_pred = XGBC.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

print((time() - t1)/60,' Minutes')

2019-12-14 12:53:14.908599
Accuracy: 28.12%
274.1999400695165  Minutes


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# split data into X and y
# grid search
model = XGBClassifier()
learning_rate = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(learning_rate=learning_rate)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=None)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
def standardize(df):
    for col in df.columns:
        mean = df[col].mean()
        std = df[col].std()
        df[col] = (df[col] - mean) / std
    
    return df

def normalize(df):
    for col in df.columns:
        minimum = df[col].min()
        maximum = df[col].max()
        df[col] = (df[col] - minimum) / (maximum - minimum)
    return df

def euclideanDist(xSeriesTestVector,xSeriesTrainVector):
    distSum = 0.0
    for index in range(len(xSeriesTestVector)):
        distSum += np.power((xSeriesTestVector[index] - xSeriesTrainVector[index]),2)
    return np.sqrt(distSum)

def manhattanDist(xSeriesTestVector,xSeriesTrainVector):
    distSum = 0.0
    for index in range(len(xSeriesTrainVector)):
        distSum += abs(xSeriesTestVector[index] - xSeriesTrainVector[index])
    return distSum

def calcDistances(xSeriesTestVector, xTrainInstances,distanceMetric):
    myArray = np.array([0.0] * len(xTrainInstances))
    for index in range(len(xTrainInstances)):
        row = xTrainInstances[index]
        myArray[index] = distanceMetric(xSeriesTestVector, row)
    return myArray

def predict(xSeriesTestVector, xTrainInstances,yTrainCategories,distanceMetric,k):
    distances = calcDistances(xSeriesTestVector, xTrainInstances,distanceMetric)
    # calculate the k- nearest neighbors, based on the above distance array
    indexArray = np.argsort(distances)
    result = 0.0
    for index in indexArray[:k]:
        result += yTrainCategories['Survived'][index]
    return np.round(result/k)

def evaluate_accuracy(xTestInstances,xTrainInstances,yTrainCategories,yTestCategories,distanceMetric,k):
    result = 0.0
    #xTestInstances=xTestInstances.reset_index(drop=True)
    #yTestCategories=yTestCategories.reset_index(drop=True)
    my_evaluate_accuracy_Array = np.array([0.0] * len(xTestInstances))
    for index in range(len(xTestInstances)):
        row = xTestInstances[index]
        my_evaluate_accuracy_Array[index] = predict(row,xTrainInstances,yTrainCategories,distanceMetric,k)
        
    for index in range(len(my_evaluate_accuracy_Array)):
        if (my_evaluate_accuracy_Array[index] == yTestCategories['Survived'][index]):
            result += 1
    return result / len(yTestCategories)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

datetime_object = datetime.datetime.now()
print(datetime_object)

myK = len(train) / 5


acc_manhattan_3 = evaluate_accuracy(X_test,X_train,y_train,y_test,manhattanDist,myK)
acc_euclidean_3 = evaluate_accuracy(X_test,X_train,y_train,y_test,euclideanDist,myK)


print ('manhattanDist,k=3, acc: %f' %(acc_manhattan_3))
print ('(Scaled) euclideanDist,k=3, acc: %f' %(acc_scaled_euclidean_3))

In [55]:
# Takes around 30-60 minutes
t1 = time()
#Printing time to know when it started
datetime_object = datetime.datetime.now()
print(datetime_object)

RFR = RandomForestRegressor()
parameters = {'n_estimators': [100],#[5,10],#,20,50,100,500],
              #'criterion': ['mse'],
              'max_depth': [15],#[5, 10, 15], 
              'min_samples_split': [100],#[2, 5, 10],
              'min_samples_leaf': [5]#[1,5]
             }

# Run the grid search
grid_obj = GridSearchCV(RFR, parameters,
                        cv=7, 
                        n_jobs=-1, 
                        verbose=1)
grid_obj = grid_obj.fit(X_train, y_train)

# Set the clf to the best combination of parameters
RFR = grid_obj.best_estimator_

# Fit the best algorithm to the data. 
RFR.fit(X_train, y_train)

#Printing time diffrence to know how much time it took
print((time() - t1)/3600,' Hours')

2019-12-12 17:03:54.925117
Fitting 7 folds for each of 1 candidates, totalling 7 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed: 27.7min finished


0.7534305562575658  Hours


In [57]:
print('----------RandomForestRegressor Classifier----------')
# make predictions for test data
y_pred = RFR.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

----------RandomForestRegressor Classifier----------
Accuracy: 27.51%


### Predict newFixtures

In [39]:
newdf = pd.DataFrame()
isHistory = False

#new Games
url = 'http://www.football-data.co.uk/fixtures.csv'
newGames = pd.read_csv(url)

#adding missing cols befor sending to games function
colList = ['HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR']
for col in colList:
    newGames[col] = 0

newGames.to_csv("fixtures.csv", index=False, sep=',')
newFixtures = pd.read_csv('fixtures.csv')

DivList = ['E0', 'E1', 'F1', 'F2', 'D1', 'D2', 'I1', 'I2', 'SP1', 'SP2']
for div in DivList:
    newGames = newFixtures[newFixtures['Div'] == div]
    newGames = newGames.fillna(0)
    games(newGames)
    newData = getNewData(newGames)
    newdf = newdf.append(newData)
newdf.to_csv("AllnewFixtureData.csv", index=False, sep=',')

In [ ]:
newdf = pd.read_csv('AllnewFixtureData.csv')
newdf

In [41]:
dropBeforPredicte = ['Date','HomeTeam','AwayTeam','FTR','HTGD','HTR', 'FTGD']
GDP = newdf.drop(dropBeforPredicte, axis=1)
GDP.apply(pd.to_numeric)
X = GDP.values
predictions = RFR.predict(X)

### Result

In [42]:
result = pd.DataFrame()
if isHistory:
    colForResult = ['Date','HomeTeam','AwayTeam','FTR', 'FTGD']#,'Referee']
else:
    colForResult = ['Date','HomeTeam','AwayTeam']#,'Referee']
result[colForResult] = newdf[colForResult]
result["GDPredictions"] = np.around(predictions) #Rounded up
result["True_GDPredictions"] = predictions
result = result.reset_index(drop=True)
result.to_csv("result.csv", index=False, sep=',')
result

,Date,HomeTeam,AwayTeam,GDPredictions,True_GDPredictions
0,09/11/2019,Burnley,West Ham,0.0,0.305772
1,09/11/2019,Chelsea,Crystal Palace,2.0,1.578443
2,09/11/2019,Leicester,Arsenal,0.0,0.392223
3,10/11/2019,Liverpool,Man City,0.0,0.104534
4,10/11/2019,Man United,Brighton,1.0,0.909869
...,...,...,...,...,...
96,09/11/2019,Las Palmas,Alcorcon,0.0,0.456026
97,10/11/2019,Malaga,Fuenlabrada,1.0,0.587884
98,09/11/2019,Mirandes,Extremadura UD,0.0,0.319785
99,10/11/2019,Racing Santander,Ponferradina,0.0,0.273582


In [ ]:
if isHistory:
    count = 0
    for i in range (len(result)):
        if(result["GDPredictions"][i] > 0 and result["FTGD"][i] > 0): 
            count+=1
        elif(result["GDPredictions"][i] == 0 and result["FTGD"][i] == 0): 
            count+=1
        elif(result["GDPredictions"][i] < 0 and result["FTGD"][i] < 0): 
            count+=1


    print("The Wining Side Predictions is Cuurect: ",count,"/",len(result))
    result.head(10)

In [ ]:
def getMachineLearningAlgorithm():
    myRFR = getRandomForestRegressor()
    myXGB = getXGBoostClassifier()
    myKNN = getKNearestNneighborsClassifier()
    return [myRFR, myXGB, myKNN]

def getAccuracy_score(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))

def getWiningSideResult(yTest, y_pred):
    count = 0
    for index in range(len(yTest)):
        if yTest[index] > 0 and y_pred[index] > 0:
            count += 1
        elif yTest[index] == 0 and y_pred[index] == 0:
            count += 1
        elif yTest[index] < 0 and y_pred[index] < 0:
            count += 1
    accuracy = count / len(yTest)
    print("The Wining Side Accuracy is: %.2f%%" % (accuracy * 100.0))
    
def getPrediction(xTest, yTest):
    df = pd.DataFrame()
    myMLA = getMachineLearningAlgorithm()
    for Algorithm in myMLA:
        AlgorithmName = str(Algorithm)
        y_pred = Algorithm(xTest)
        df[AlgorithmName] = y_pred
        df[AlgorithmName + ' rounded'] = np.around(result)
        if(yTest != None):
            # evaluate predictions
            print(str(Algorithm))
            getAccuracy_score(y_test, y_pred)
            getWiningSideResult(yTest, y_pred)
            
    return df

def getGamesToPredict(isNew):
    #TODO Read From File
    if isNew:
        getFromSite()
    else:
        readFromFile()
    return xTest, yTest

def getInfoColumnsNames():
    return ['Date', 'HomeTeam', 'AwayTeam']

def getResultCsv(csvName):
    isNew = True
    df = pd.DataFrame()
    infoColumns = getInfoColumnsNames()
    xTest, yTest = getGamesToPredict(isNew)
    pred_df = getPrediction(xTest, yTest)
    
    for col in infoColumns:
        df[col] = xTest[col]
        
    for col in pred_df:
        df[col] = pred_df[col]
    
    #df = df.reset_index(drop=True)
    df.to_csv(csvName + ".csv", index=False, sep=',')
    print(df)